# Point matching

Matching legs that start and end in similar points

In [ ]:
# Import libraries

import os
import sys
import csv
import json
import time
import pathlib
import itertools
from datetime import date, datetime
from multiprocessing import Pool

# numerical libraries
import pandas as pd
import numpy as np

# plotting libraries
import seaborn as sns
import matplotlib
import matplotlib.pyplot as plt
from matplotlib import rcParams

In [ ]:
# graphical parameters
rcParams["axes.titlepad"] = 45
rcParams["font.size"] = 16
rcParams["figure.figsize"] = 12, 8
sns.set_style("whitegrid")

In [ ]:
# global variables
cutting_date = "2019-05-01"  # remove trips and data published before this date
meta_data_path = pathlib.Path("../../data-campaigns/meta-data/")
input_path = pathlib.Path("../../2019-12-16.out/")
out_path = pathlib.Path("../../2019-12-16.out/matching_points")

In [ ]:
try:
    os.makedirs(os.path.abspath(out_path))
except FileExistsError:
    print("Directory '{}' already exists".format(out_path), file=sys.stderr)

In [ ]:
# input files
legs = "all_legs_merged_no_outlier_0.01.pkl"
trips_users = "trips_users_df.pkl"
trips = "trips_df.pkl"
users_with_trips = "users_df_with_trips.pkl"

# read datasets
legs_df = pd.read_pickle(input_path / legs)
trips_users_df = pd.read_pickle(input_path / trips_users)
trips_df = pd.read_pickle(input_path / trips)
users_df_with_trips = pd.read_pickle(input_path / users_with_trips)

In [ ]:
all_legs_coords_filename = "all_legs_final_ds_user_info_urban_class.pkl"
all_legs_coords = pd.read_pickle(os.path.join(input_path, all_legs_coords_filename))

In [ ]:
gps_cities_filename = "gps_cities.pkl"
gps_cities = pd.read_pickle(os.path.join(input_path, gps_cities_filename))

In [ ]:
gps_cities.head(3)

In [ ]:
gps_cities.columns

In [ ]:
legs_coords_df = gps_cities[
    [
        "legid",
        "StartLat",
        "StartLon",
        "country_start",
        "start_class",
        "EndLat",
        "EndLon",
        "country_end",
        "end_class",
    ]
]
legs_coords_df = legs_coords_df.rename(
    columns={
        "StartLat": "lat_start",
        "StartLon": "lon_start",
        "start_class": "class_start",
        "EndLat": "lat_end",
        "EndLon": "lon_end",
        "end_class": "class_end",
    }
)
legs_coords_df.drop_duplicates(keep="first", inplace=True)
legs_coords_df.head(3)

## Coordinate rounding procedure

Following the information on the Wikipedia page [Decimal degree](https://en.wikipedia.org/w/index.php?title=Decimal_degrees&oldid=937245621#Precision) and the question on StackOverflow ["Measuring accuracy of latitude and longitude?"](https://gis.stackexchange.com/q/8650/18292), we have that:
> The third decimal place is worth up to 110 m: it can identify a large agricultural field or institutional campus.
```
3        0.001            111  m
```

We will proceed like this: we consider each point (lat, lon) to be represented by a square given with the following vertices:
* `A (lat-0.002, lon+0.002)`
* `B (lat+0.002, lon+0.002)`
* `C (lat+0.002, lon-0.002)`
* `D (lat-0.002, lon-0.002)`

In this way each point is effectively transformed in a square - or, rather a curved square, each side is an arc - with sides of lenght o.004 degrees.

If the the squares representing two points intersect we consider them equal. In this way two points are distant at most:
```
sqrt(2)·(0.004 deg)·(111.32 km/deg) = 629,72102 m ~ 630 m
```

Graphical example:
<img src="https://i.imgur.com/fSh5ISh.png" />

In [ ]:
EPS = 0.002

In [ ]:
subdir = "eps_{}".format(EPS)
out_subdir_path = out_path / subdir

In [ ]:
try:
    os.makedirs(os.path.abspath(out_subdir_path))
except FileExistsError:
    print("Directory '{}' already exists".format(out_subdir_path), file=sys.stderr)

In [ ]:
# Find if two rectangles overlap
# https://www.geeksforgeeks.org/find-two-rectangles-overlap/


class Point:
    def __init__(self, x, y):
        self.x = x
        self.y = y


# Returns true if two rectangles(l1, r1) and (l2, r2) overlap
def rect_overlap(l1, r1, l2, r2):
    # If one rectangle is on left side of other
    if l1.x > r2.x or l2.x > r1.x:
        return False

    # If one rectangle is above other
    if l1.y < r2.y or l2.y < r1.y:
        return False

    return True


def equivalent_points(p1lat, p1lon, p2lat, p2lon):
    # A (lat-0.002, lon+0.002)
    # B (lat+0.002, lon+0.002)
    # C (lat+0.002, lon-0.002)
    # D (lat-0.002, lon-0.002)

    a1 = Point(p1lat - EPS, p1lon + EPS)
    b1 = Point(p1lat + EPS, p1lon + EPS)
    c1 = Point(p1lat + EPS, p1lon - EPS)
    d1 = Point(p1lat - EPS, p1lon - EPS)

    a2 = Point(p2lat - EPS, p2lon + EPS)
    b2 = Point(p2lat + EPS, p2lon + EPS)
    c2 = Point(p2lat + EPS, p2lon - EPS)
    d2 = Point(p2lat - EPS, p2lon - EPS)

    return rect_overlap(a1, c1, a2, c2)

In [ ]:
# tests
assert equivalent_points(1.0, 1.0, 1.00405, 1.004) is False
assert equivalent_points(1.0, 1.0, 1.001, 1.001) is True

In [ ]:
(
    legs_coords_df[
        ["legid", "country_start", "country_end", "class_start", "class_end"]
    ]
    .fillna("NONE")
    .groupby(["country_start", "country_end", "class_start", "class_end"])
    .size()
    .sort_values(ascending=False)
    .reset_index()
).head(10)

In [ ]:
legs_coords_df.groupby(["country_start"]).size().sort_values(
    ascending=False
).reset_index().head(10)

In [ ]:
legs_coords_df.groupby(["country_start", "class_start"]).size().sort_values(
    ascending=False
).reset_index().head(10)

In [ ]:
legs_coords_df.groupby(["country_end", "class_end"]).size().sort_values(
    ascending=False
).reset_index().head(10)

In [ ]:
countries = set(legs_coords_df.country_start.fillna("NONE").unique()).union(
    set(legs_coords_df.country_end.fillna("NONE").unique())
)
countries.discard("NONE")
print("Number of different countries:", len(countries))
print(countries)

point_classes = set(legs_coords_df.class_start.fillna("NONE").unique()).union(
    set(legs_coords_df.class_end.fillna("NONE").unique())
)
point_classes.discard("NONE")
print("Number of classes:", len(point_classes))
print(point_classes)

In [ ]:
legs_coords_df.columns

In [ ]:
PRINT_NROWS = 1000000


def select_legs(coords1_df, coords2_df, country, pc, match_type):
    tmp1_df = coords1_df.loc[
        ((coords1_df["country"] == country) | (coords1_df["country"] == "NONE"))
        & ((coords1_df["class"] == pc) | (coords1_df["class"] == "NONE"))
    ].drop_duplicates()

    tmp2_df = coords2_df.loc[
        ((coords2_df["country"] == country) | (coords2_df["country"] == "NONE"))
        & ((coords2_df["class"] == pc) | (coords2_df["class"] == "NONE"))
    ].drop_duplicates()

    npoints1 = tmp1_df.legid.nunique()
    npoints2 = tmp2_df.legid.nunique()
    print(
        "- ({}, {}, {}) Points 1: {}, Points 2: {}, To Process: {} - ".format(
            country, pc, match_type, npoints1, npoints2, npoints1 * npoints2
        ),
        end="",
    )

    if npoints1 > 0 and npoints2 > 0:
        i = 0
        # iterating over multiple columns
        for row1 in tmp1_df.itertuples():
            for row2 in tmp2_df.itertuples():
                i = i + 1
                if (i % PRINT_NROWS) == 0:
                    print(".", end="")
                if (i % (10 * PRINT_NROWS)) == 0:
                    print(" ", end="")

                # equivalent_points(p1lat, p1lon, p2lat, p2lon):
                if row1.legid > row2.legid and equivalent_points(
                    row1.lat, row1.lon, row2.lat, row2.lon
                ):
                    yield (row1.legid, row2.legid)

    print()

In [ ]:
legs_start_coords_df = legs_coords_df[
    ["legid", "lat_start", "lon_start", "country_start", "class_start"]
].copy()
legs_start_coords_df["country_start"] = legs_start_coords_df["country_start"].fillna(
    "NONE"
)
legs_start_coords_df["class_start"] = legs_start_coords_df["class_start"].fillna("NONE")
legs_start_coords_df.columns = ["legid", "lat", "lon", "country", "class"]

legs_end_coords_df = legs_coords_df[
    ["legid", "lat_end", "lon_end", "country_end", "class_end"]
].copy()
legs_end_coords_df["country_end"] = legs_end_coords_df["country_end"].fillna("NONE")
legs_end_coords_df["class_end"] = legs_end_coords_df["class_end"].fillna("NONE")
legs_end_coords_df.columns = ["legid", "lat", "lon", "country", "class"]

In [ ]:
OUTFILE_BASENAME = "matching_points"


def process(country_pc):
    country = country_pc[0]
    pc = country_pc[1]
    print("Processing: {} ({})".format(country, pc))

    outfile_name = "{}_{}_{}.csv".format(OUTFILE_BASENAME, country, pc)
    outfile_path = out_subdir_path / outfile_name
    with open(outfile_path, "w+") as outfp:
        writer = csv.writer(outfp)

        writer.writerow(("legid1", "class1", "legid2", "class2"))

        for match in select_legs(
            legs_start_coords_df, legs_start_coords_df, country, pc, "start-start"
        ):
            legid1 = match[0]
            legid2 = match[1]

            writer.writerow((legid1, "start", legid2, "start"))

        for match in select_legs(
            legs_start_coords_df, legs_end_coords_df, country, pc, "start-end"
        ):
            legid1 = match[0]
            legid2 = match[1]

            writer.writerow((legid1, "start", legid2, "end"))

        for match in select_legs(
            legs_end_coords_df, legs_end_coords_df, country, pc, "end-end"
        ):
            legid1 = match[0]
            legid2 = match[1]

            writer.writerow((legid1, "end", legid2, "end"))

In [ ]:
args = [el for el in itertools.product(sorted(countries), sorted(point_classes))]
print("# of args:", len(args))
print("args:", args)

In [ ]:
# cell tagged with 'parameters'
NPROCS = 4

In [ ]:
print("Number of subprocesses to launch:", NPROCS)

In [ ]:
%%time
with Pool(NPROCS) as p:
    p.map(process, args)